**The code below will take the CDEM, reproject it to EPSG 2958 (UTM 17N) and resample it to 30m, creating a new elevation raster in a new folder that can be used to derive other features**

In [1]:
import os

# Paths
dem_path = r'DEM.tif'
reference_tif = r'Mosaicked and Clipped Outputs/alpha_orf_mosaicked_clip.tif'  # Provide the path to your reference GeoTIFF
output_folder = r'DEM Outputs'

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Output path
output_dem_path = os.path.join(output_folder, "elevation.tif")

# Reproject the DEM to EPSG 2958, set the resolution, and align to a grid without clipping
reproject_cmd = f'gdalwarp -s_srs EPSG:4326 -t_srs EPSG:2958 -tr 30 30 -tap -r cubic -overwrite -of GTiff "{dem_path}" "{output_dem_path}"'
os.system(reproject_cmd)

# Print message
print("\nReprojected and resampled DEM created at:", output_dem_path)


Creating output file that is 9137P x 8562L.
Processing DEM.tif [1/1] : 0Using internal nodata values (e.g. -9999) for image DEM.tif.
Copying nodata values from source DEM.tif to destination DEM Outputs/elevation.tif.
...10...20...30...40...50...60...70...80...90...100 - done.

Reprojected and resampled DEM created at: DEM Outputs/elevation.tif




**Calculate Slope and Aspect**

In [2]:
from osgeo import gdal
import os

def calculate_slope_aspect(dem_path):
    try:
        # Extract the folder path and filename from the input DEM path
        dem_folder, dem_filename = os.path.split(dem_path)

        # Create the output folder if it doesn't exist
        output_folder = os.path.join(dem_folder)
        os.makedirs(output_folder, exist_ok=True)

        # Paths for final output
        slope_path = os.path.join(output_folder, "slope.tif")
        aspect_path = os.path.join(output_folder, "aspect.tif")

        # Calculate slope using gdaldem utility
        gdal.DEMProcessing(slope_path, dem_path, 'slope', format='GTiff')

        # Calculate aspect using gdaldem utility
        gdal.DEMProcessing(aspect_path, dem_path, 'aspect', format='GTiff')

        print("\nSLOPE AND ASPECT RASTERS SAVED\n")

    except Exception as e:
        print(f"Error: {e}")

# Specify the path to the DEM file
dem_path = "DEM Outputs/elevation.tif"

# Run the function
calculate_slope_aspect(dem_path)

print("DONE!")


/Users/maciej/miniconda3/envs/envmod/lib/python3.12/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(



SLOPE AND ASPECT RASTERS SAVED

DONE!


**Perform a clip operation on all .tif files in the folder**

In [7]:
import os
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.crs import CRS

def clip_rasters_to_shapefile(input_folder, shapefile_path):
    # Get a list of all .tif files in the input folder
    tif_files = [f for f in os.listdir(input_folder) if f.endswith('.tif')]

    total_files = len(tif_files)

    print(f"Clipping {total_files} rasters:")
    for i, tif_file in enumerate(tif_files):
        input_path = os.path.join(input_folder, tif_file)

        # Open the raster dataset
        with rasterio.open(input_path) as src:
            # Read the shapefile
            shapefile = gpd.read_file(shapefile_path)
            
            # Check and set CRS if needed
            if not shapefile.crs or shapefile.crs != CRS.from_string(src.crs.to_string()):
                shapefile = shapefile.to_crs(src.crs)

            # Get raster bounds
            raster_bounds = src.bounds

            # Clip shapefile to raster bounds
            shapefile = shapefile.cx[raster_bounds.left:raster_bounds.right, raster_bounds.bottom:raster_bounds.top]

            # Check if there are any geometries left after clipping
            if shapefile.empty:
                print(f"No overlapping geometries found for {tif_file}. Skipping...")
                continue

            shapes = shapefile.geometry.tolist()

            # Clip the raster using the shapefile and set values outside shapefile boundaries to -9999
            out_image, out_transform = mask(src, shapes, crop=True, nodata=-9999)

            # Update metadata
            out_meta = src.meta.copy()
            out_meta.update({"driver": "GTiff",
                             "height": out_image.shape[1],
                             "width": out_image.shape[2],
                             "transform": out_transform,
                             "nodata": -9999})

            # Write the clipped raster to disk
            output_path = os.path.join(output_folder, f"{tif_file}")
            with rasterio.open(output_path, "w", **out_meta) as dest:
                dest.write(out_image)

        # Print progress information
        print(f"Processed {i + 1} of {total_files} files")

if __name__ == "__main__":
    # Replace these paths with your actual paths
    input_folder = "DEM Outputs"
    shapefile_path = "Algonquin Park Shapefile/Algonquin_Park_Final.shp"

    # Run the function
    clip_rasters_to_shapefile(input_folder, shapefile_path)


Clipping 3 rasters:
Processed 1 of 3 files
Processed 2 of 3 files
Processed 3 of 3 files
